In [20]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [21]:
df = pd.read_csv("data/100x100.csv")
data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
print(data_matrix.shape)


(98, 97)


In [22]:
delta=df[['u_id', 'a_id', 'score']]
delta

,u_id,a_id,score
0,0,31240,9
1,16,31240,8
2,31,31240,9
3,50,31240,9
4,77,31240,4
...,...,...,...
1786,1098,28623,8
1787,1349,28623,8
1788,1484,28623,4
1789,3413,28623,6


In [23]:
user_id_to_index = {} #helper function
for idx in range(len(data_matrix.index)):
    u_id = data_matrix.index[idx]
    user_id_to_index[u_id] = idx
    
# user_id_to_index[5]

In [24]:
anime_id_to_index = {} #helper function
for idx in range(len(data_matrix.columns)):
    a_id = data_matrix.columns[idx]
    anime_id_to_index[a_id] = idx

# anime_id_to_index[35760]

In [25]:
# user_id_to_index[1892]

In [26]:
import concurrent.futures

def spectral_regularization_model(x):
    delta = df[['u_id', 'a_id', 'score']]
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    R = cp.Variable(shape=(num_users, num_anime))

    def compute_error(ui):
        u = user_id_to_index[int(delta.at[ui, 'u_id'])]
        i = anime_id_to_index[int(delta.at[ui, 'a_id'])]
        return (R[u][i] - int(delta.at[ui, 'score'])) ** 2

    with concurrent.futures.ThreadPoolExecutor() as executor:
        training_error = list(executor.map(compute_error, range(len(delta))))

    training_error_sum = cp.sum(training_error)
    print("training_error_sum:", training_error_sum)

    obj = cp.Minimize(training_error_sum + x * cp.norm(R, "nuc"))
    prob = cp.Problem(obj)
    prob.solve()
    print("status:", prob.status)


In [27]:
spectral_regularization_model(10000)


training_error_sum: power(var619959[0, 0:97][70] + -9.0, 2.0) + power(var619959[2, 0:97][70] + -8.0, 2.0) + power(var619959[3, 0:97][70] + -9.0, 2.0) + power(var619959[7, 0:97][70] + -9.0, 2.0) + power(var619959[10, 0:97][70] + -4.0, 2.0) + power(var619959[11, 0:97][70] + -9.0, 2.0) + power(var619959[12, 0:97][70] + -10.0, 2.0) + power(var619959[18, 0:97][70] + -8.0, 2.0) + power(var619959[25, 0:97][70] + -8.0, 2.0) + power(var619959[31, 0:97][70] + -8.0, 2.0) + power(var619959[33, 0:97][70] + -6.0, 2.0) + power(var619959[34, 0:97][70] + -2.0, 2.0) + power(var619959[35, 0:97][70] + -6.0, 2.0) + power(var619959[41, 0:97][70] + -8.0, 2.0) + power(var619959[57, 0:97][70] + -5.0, 2.0) + power(var619959[58, 0:97][70] + -4.0, 2.0) + power(var619959[59, 0:97][70] + -5.0, 2.0) + power(var619959[63, 0:97][70] + -3.0, 2.0) + power(var619959[64, 0:97][70] + -5.0, 2.0) + power(var619959[66, 0:97][70] + -8.0, 2.0) + power(var619959[67, 0:97][70] + -8.0, 2.0) + power(var619959[68, 0:97][70] + -8.0, 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


status: optimal
